In [1]:
import pandas as pd
import numpy as np
from buckaroo.buckaroo_widget import BuckarooWidget

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
df = pd.read_csv('/Users/paddy/code/citibike-play/2014-01 - Citi Bike trip data.csv')
BuckarooWidget(df[:10_000])

BuckarooWidget(commandConfig={'argspecs': {'dropcol': [None], 'to_datetime': [None], 'safeint': [None], 'filln…

In [ ]:
def sample(df, sample_size=500, include_outliers=True):
    
    sample_size = np.min([sample_size, len(df)])
    sdf = df.sample(sample_size)
    
    
    if include_outliers:
        outlier_idxs = []
        for col in df.columns:
            idxs = df[col].sort_values().index
            outlier_idxs.extend(idxs[:5])
            outlier_idxs.extend(idxs[-5:])
        outlier_idxs.extend(sdf.index)
        uniq_idx = np.unique(outlier_idxs)
        #print(uniq_idx)
        return df.iloc[uniq_idx]
    return sdf


In [ ]:
%timeit sample(df, 500)

In [ ]:
%timeit sample(df, 10)

In [ ]:
%timeit sample(df, 5000)

In [ ]:
len(sample(df))

In [ ]:
df.iloc[[23,58, 1023]]

In [ ]:
import numpy as np

In [ ]:
df = pd.read_csv('./examples/data/2014-01-citibike-tripdata.csv')
w = BuckarooWidget(df)
w

In [ ]:
ab = df['tripduration']
ab.sort_values()

In [ ]:
sorted_ab = ab.sort_values()
sorted_ab.index[:5]

In [ ]:
import numpy as np

In [ ]:
np.unique([2,2,3])

In [ ]:
df['start station name'].sort_values()

In [ ]:
np.array([2,2,3]).unique()

In [ ]:
a = []
a.extend([23,45], [32,5])
a

In [ ]:
df.columns

In [ ]:
bc = df.sample(300)

In [ ]:
bc.index

In [ ]:
arr = [2,3]
arr.extend(df.index)

In [ ]:
df.iloc[[137, 137, 138]]

In [ ]:
pd.concat([df.iloc[sorted_ab.index[:5]], df[20:50]])

In [ ]:
from buckaroo.all_transforms import Command
from buckaroo.lispy import s
#Here we start adding commands to the Buckaroo Widget.  Every call to add_command replaces a command with the same name
@w.add_command
class GroupBy2(Command):
    command_default = [s("groupby2"), s('df'), 'col', {}]
    command_pattern = [[3, 'colMap', 'colEnum', ['null', 'sum', 'mean', 'median', 'count']]]
    @staticmethod 
    def transform(df, col, col_spec):
        grps = df.groupby(col)
        df_contents = {}
        for k, v in col_spec.items():
            if v == "sum":
                df_contents[k] = grps[k].apply(lambda x: x.sum())
            elif v == "mean":
                df_contents[k] = grps[k].apply(lambda x: x.mean())
            elif v == "median":
                df_contents[k] = grps[k].apply(lambda x: x.median())
            elif v == "count":
                df_contents[k] = grps[k].apply(lambda x: x.count())
        return pd.DataFrame(df_contents)

    @staticmethod 
    def transform_to_py(df, col, col_spec):
        commands = [
            "    grps = df.groupby('%s')" % col,
            "    df_contents = {}"
        ]
        for k, v in col_spec.items():
            if v == "sum":
                commands.append("    paddydf_contents['%s'] = grps['%s'].apply(lambda x: x.sum())" % (k, k))
            elif v == "mean":
                commands.append("    df_contents['%s'] = grps['%s'].apply(lambda x: x.mean())" % (k, k))
            elif v == "median":
                commands.append("    df_contents['%s'] = grps['%s'].apply(lambda x: x.median())" % (k, k))
            elif v == "count":
                commands.append("    df_contents['%s'] = grps['%s'].apply(lambda x: x.count())" % (k, k))
        commands.append("    df = pd.DataFrame(df_contents)")
        return "\n".join(commands)
